In [1]:
# Mounting the google drive to get the images

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
# Importing some useful libraries and scripts

import pandas as pd # For dataframe related tasks
import numpy as np # For numerical tasks

import pickle # For model saving

# For train test spliting while training our model
from sklearn.model_selection import train_test_split

# For data preporcessing
from sklearn.preprocessing import StandardScaler

# A module for model training
from sklearn import svm

# A module for accuracy score checking
from sklearn.metrics import accuracy_score

In [3]:
# Specifying the location for all our data is located

data_folder_path = 'drive/MyDrive/AI_project_data'

In [4]:
# Checking our data

df = pd.read_csv(data_folder_path + '/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# Lets check the mean standard deviation and other statstical measures of our dataset

df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
# Checking to see if there is any categorical column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
# Lets see how different features are related to the outcome
# 0 is a person that is diabetes free
# 1 is a person with diabetes

df.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [8]:
# Since from the dataset we have many range of values so we need to do some scaling

scaler = StandardScaler()
X = scaler.fit_transform(np.array(df.drop('Outcome',axis = 1)))
y = df['Outcome']
print(X.shape)
print(y.shape)

(768, 8)
(768,)


In [9]:
'''
 Train test splitting using (80,20) ratio
 We used the argument stratify to have a standardized train test split
 i.e. the train set will have a balanced 0 and 1 ratio
'''

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    stratify = y,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [10]:
# Instantiating and training the supported vector classifier model

classifier = svm.SVC(kernel='linear')
classifier.fit(X_train,y_train)

SVC(kernel='linear')

In [11]:
# Checking the training set accuracy 

pred = classifier.predict(X_train)
accuracy = round(accuracy_score(pred,y_train),2)
print(f'Accuracy of the training set is: {int(accuracy * 100)}%')

Accuracy of the training set is: 79%


In [12]:
# Checking the test set accuracy to check if there is any overfitting

pred = classifier.predict(X_test)
accuracy = round(accuracy_score(pred,y_test),2)
print(f'Accuracy of the test set is: {int(accuracy * 100)}%')

Accuracy of the test set is: 72%


In [13]:
# Lets take some values in the test set and predict

check_data = X_test[:5]
prediction = classifier.predict(check_data)

In [14]:
# Checking the accuracy of our sampled data

accurate_results = np.array(prediction == y_test[:5],dtype=int)
print('The model accurately predicted {} out of {}'.format(
    np.sum(accurate_results),
    len(accurate_results)
))

The model accurately predicted 3 out of 5


In [17]:
# Saving the model 

file_name = 'models/diabetes_model.h5'
with open(file_name,'wb') as f:
  pickle.dump(classifier,f)